In [1]:
import numpy as np
import mne
import os
from scipy.spatial.distance import cdist

In [2]:


def get_neighbors_within_radius(pos:list, radius:float) -> list:
    """
    获取通道位置后输入该函数，设定sphere半径
    计算在sphere中的channel的索引并返回

    Parameters
    ----------
    pos: 所有通道的三维坐标
    radius: searchlight sphere半径


    Returns
    -------

    neighbors: 每个sphere内的mag索引组成的嵌套列表
    """
    dist_mat = cdist(pos, pos)
    neighbors = [np.where(dist_mat[i] <= radius)[0].tolist() for i in range(len(pos))]

    return neighbors

def get_chanpo_fif(subj:int, radius: float = .04):
    """
    通过被试的epoch信息，提取以每个channel为中心的sphere内mag、grad索引

    Parameters
    ----------
    subj: 单一被试索引
    radius: searchlight sphere半径，默认值0.04

    Returns
    -------
    mag_idx_idx: list, 每个sphere内的mag索引组成的嵌套列表
    grad_idx_idx: list, 每个sphere内的grad索引组成的嵌套列表

    """
    datadir = r'../data'
    sub_fif_dir = r'cleaned/unconscious-session_1-block_1-epo.fif'
    epo_fif_dir = os.path.join(datadir,f'subject{subj}',sub_fif_dir)
    epo_fif = mne.read_epochs(epo_fif_dir, preload=True)

    # 提取磁强计索引
    mag_idx = mne.pick_types(epo_fif.info, meg='mag')

    # 提取梯度计索引
    grad_idx = mne.pick_types(epo_fif.info, meg='grad')

    # 获取磁强计的位置
    mag_pos = [epo_fif.info['chs'][i]['loc'][:3] for i in mag_idx]

    # 获取梯度计的位置
    grad_pos = [epo_fif.info['chs'][i]['loc'][:3] for i in grad_idx]


    mag_idx_idx = get_neighbors_within_radius(mag_pos, radius)

    # 判断sphere中是否只有中心点
    for idx in range( len( mag_idx_idx ) ):
        if len( mag_idx_idx[idx] )  <2:
            raise ValueError("radius值过小")

    grad_idx_idx = np.arange(len(grad_pos)).reshape(-1,2)
    grad_idx_idx = [grad_idx_idx[item].flatten() for item in mag_idx_idx]

    # 判断sphere中是否只有中心点
    for idx in range( len( grad_idx_idx ) ) :
        if len( grad_idx_idx[idx] ) <4:
            raise ValueError("radius值过小")

    return mag_idx_idx, grad_idx_idx

In [4]:
mag_idx_idx, grad_idx_idx = get_chanpo_fif(17,radius = .04)


Reading D:\Archive\Github\COSN-Hackathon2025\scripts\..\data\subject17\cleaned\unconscious-session_1-block_1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...     998.00 ms
        0 CTF compensation matrices available
Not setting metadata
98 matching events found
No baseline correction applied
0 projection items activated


In [5]:
mag_idx_idx

[[0, 1, 2, 3],
 [0, 1, 2, 11],
 [0, 1, 2, 3, 4, 54],
 [0, 2, 3, 54, 57],
 [2, 4, 5, 7, 11],
 [4, 5, 6, 9, 12],
 [5, 6, 7, 15, 59],
 [4, 6, 7, 54, 58],
 [8, 9, 11, 16],
 [5, 8, 9, 10, 11, 19],
 [9, 10, 12, 23],
 [1, 4, 8, 9, 11],
 [5, 10, 12, 13, 15],
 [12, 13, 14, 22, 23],
 [13, 14, 15, 24, 67],
 [6, 12, 14, 15, 66],
 [8, 16, 17, 19],
 [16, 17, 18, 28],
 [17, 18, 19, 20, 29],
 [9, 16, 18, 19],
 [18, 20, 23, 34],
 [21, 22, 23, 34, 36, 37],
 [13, 21, 22, 23, 24, 37],
 [10, 13, 20, 21, 22, 23],
 [14, 22, 24, 25, 27],
 [24, 25, 26, 37, 41],
 [25, 26, 27, 82, 85],
 [24, 26, 27, 67, 68],
 [17, 28, 29, 30],
 [18, 28, 29, 33, 34],
 [28, 30, 31, 33],
 [30, 31, 32, 42],
 [31, 32, 33, 44],
 [29, 30, 32, 33, 35],
 [20, 21, 29, 34, 35],
 [33, 34, 35, 36],
 [21, 35, 36, 37, 38, 45],
 [21, 22, 25, 36, 37, 38],
 [36, 37, 38, 39, 41],
 [38, 39, 40, 45, 46],
 [39, 40, 41, 49, 83],
 [25, 38, 40, 41, 82],
 [31, 42, 43, 44],
 [42, 43, 44, 47, 50],
 [32, 42, 43, 44, 45, 46],
 [36, 39, 44, 45],
 [39, 44, 46,

In [150]:
grad_idx_idx

[array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 0,  1,  2,  3,  4,  5, 22, 23]),
 array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9, 108, 109]),
 array([  0,   1,   4,   5,   6,   7, 108, 109, 114, 115]),
 array([ 4,  5,  8,  9, 10, 11, 14, 15, 22, 23]),
 array([ 8,  9, 10, 11, 12, 13, 18, 19, 24, 25]),
 array([ 10,  11,  12,  13,  14,  15,  30,  31, 118, 119]),
 array([  8,   9,  12,  13,  14,  15, 108, 109, 116, 117]),
 array([16, 17, 18, 19, 22, 23, 32, 33]),
 array([10, 11, 16, 17, 18, 19, 20, 21, 22, 23, 38, 39]),
 array([18, 19, 20, 21, 24, 25, 46, 47]),
 array([ 2,  3,  8,  9, 16, 17, 18, 19, 22, 23]),
 array([10, 11, 20, 21, 24, 25, 26, 27, 30, 31]),
 array([24, 25, 26, 27, 28, 29, 44, 45, 46, 47]),
 array([ 26,  27,  28,  29,  30,  31,  48,  49, 134, 135]),
 array([ 12,  13,  24,  25,  28,  29,  30,  31, 132, 133]),
 array([16, 17, 32, 33, 34, 35, 38, 39]),
 array([32, 33, 34, 35, 36, 37, 56, 57]),
 array([34, 35, 36, 37, 38, 39, 40, 41, 58, 59]),
 array([18, 19, 32, 33, 36, 

In [113]:
# mag_idx_idx
grad_idx_idx

[array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 0,  1,  2,  3,  4,  5, 22, 23]),
 array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9, 108, 109]),
 array([  0,   1,   4,   5,   6,   7, 108, 109, 114, 115]),
 array([ 4,  5,  8,  9, 10, 11, 14, 15, 22, 23]),
 array([ 8,  9, 10, 11, 12, 13, 18, 19, 24, 25]),
 array([ 10,  11,  12,  13,  14,  15,  30,  31, 118, 119]),
 array([  8,   9,  12,  13,  14,  15, 108, 109, 116, 117]),
 array([16, 17, 18, 19, 22, 23, 32, 33]),
 array([10, 11, 16, 17, 18, 19, 20, 21, 22, 23, 38, 39]),
 array([18, 19, 20, 21, 24, 25, 46, 47]),
 array([ 2,  3,  8,  9, 16, 17, 18, 19, 22, 23]),
 array([10, 11, 20, 21, 24, 25, 26, 27, 30, 31]),
 array([24, 25, 26, 27, 28, 29, 44, 45, 46, 47]),
 array([ 26,  27,  28,  29,  30,  31,  48,  49, 134, 135]),
 array([ 12,  13,  24,  25,  28,  29,  30,  31, 132, 133]),
 array([16, 17, 32, 33, 34, 35, 38, 39]),
 array([32, 33, 34, 35, 36, 37, 56, 57]),
 array([34, 35, 36, 37, 38, 39, 40, 41, 58, 59]),
 array([18, 19, 32, 33, 36, 

In [155]:
get_chanpo_fif(17,0.01)

Reading D:\Archive\Github\COSN-Hackathon2025\scripts\..\data\subject17\cleaned\unconscious-session_1-block_1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...     998.00 ms
        0 CTF compensation matrices available
Not setting metadata
98 matching events found
No baseline correction applied
0 projection items activated


ValueError: radius值过小

In [67]:
subj = 17
datadir = r'../data'
sub_fif_dir = r'cleaned/unconscious-session_1-block_1-epo.fif'
epo_fif_dir = os.path.join(datadir,f'subject{subj}',sub_fif_dir)
epo_fif = mne.read_epochs(epo_fif_dir, preload=True)
grad = epo_fif.copy().pick(picks='grad')
print(grad.ch_names[:10])
one_grad = grad.copy().pick_channels([grad.ch_names[0]])
two_grad = grad.copy().pick_channels([grad.ch_names[1]])
# grad: 只包含梯度计
# epo_fif: 仍然保留所有通道


Reading D:\Archive\Github\COSN-Hackathon2025\scripts\..\data\subject17\cleaned\unconscious-session_1-block_1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...     998.00 ms
        0 CTF compensation matrices available
Not setting metadata
98 matching events found
No baseline correction applied
0 projection items activated
['MEG0112', 'MEG0113', 'MEG0122', 'MEG0123', 'MEG0132', 'MEG0133', 'MEG0142', 'MEG0143', 'MEG0212', 'MEG0213']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [70]:
%matplotlib notebook


a=one_grad.get_data()
a.shape
b=two_grad.get_data()

print(a),print(''),

[[[ 3.74903831e-13  5.11195977e-13  8.95155441e-14 ... -4.85940974e-12
   -4.35796780e-12 -3.73933384e-12]]

 [[-2.49354241e-13  9.54754991e-13  1.68761499e-12 ...  3.50732404e-12
    1.56638656e-12 -7.56091536e-13]]

 [[-4.53442671e-12 -4.26807586e-12 -3.81078214e-12 ...  2.29370293e-12
    1.17673961e-12  1.50123421e-12]]

 ...

 [[-1.35120045e-12 -6.82906153e-13 -3.89456515e-13 ... -2.40964053e-12
   -4.14541373e-12 -4.27106052e-12]]

 [[ 4.84782477e-12  3.82662639e-12  2.77745036e-12 ...  4.46088249e-12
    6.15130515e-12  6.99256408e-12]]

 [[-3.74349954e-12 -3.81857726e-12 -3.89465007e-12 ...  2.15885758e-12
    1.54377039e-12 -9.06362062e-13]]]

[[[ 1.22292595e-12  8.83904649e-13  4.67807853e-13 ... -8.99858510e-13
   -1.85559393e-12 -4.08110303e-12]]

 [[-2.40165323e-12 -6.55412089e-14  2.63233812e-12 ... -7.20774822e-12
   -9.10750724e-12 -8.09188805e-12]]

 [[-3.61754800e-12 -7.74894116e-12 -9.40038732e-12 ...  1.42417405e-12
    5.78762505e-14  5.07014248e-13]]

 ...

 [[ 5.

(None, None, None)

In [71]:
print(b)

[[[ 1.22292595e-12  8.83904649e-13  4.67807853e-13 ... -8.99858510e-13
   -1.85559393e-12 -4.08110303e-12]]

 [[-2.40165323e-12 -6.55412089e-14  2.63233812e-12 ... -7.20774822e-12
   -9.10750724e-12 -8.09188805e-12]]

 [[-3.61754800e-12 -7.74894116e-12 -9.40038732e-12 ...  1.42417405e-12
    5.78762505e-14  5.07014248e-13]]

 ...

 [[ 5.05721354e-14 -3.74621104e-13  5.92791343e-13 ... -3.59850397e-12
   -8.08261655e-12 -1.15494588e-11]]

 [[ 5.83196512e-12  6.32443485e-12  5.94517010e-12 ...  1.42069276e-12
   -1.64087409e-14 -6.44987183e-13]]

 [[-1.27937446e-12 -4.77889594e-12 -7.00073080e-12 ... -2.74804890e-12
   -2.54332186e-12  1.65195875e-13]]]


In [73]:
print(np.shape(grad.ch_names))

(204,)
